In [ ]:
#start with a 5 loc instance and work from there
#25 xs
# (a,1),(b,2),(c,3),(d,4),(e,5)
# sum fij dkp xik xjp
# fab d12 xa1 xb2

def evaluate(assign, size):
    total = 0
    for i in range(size):
        for j in range(size):
            #if i!=j:
            total += flows[i][j]*distances[assign[i]][assign[j]]
    return total
def local_search_first(val, curr, size):
    for i in range(size):
        for j in range(size):
            if i<j:
                temp = curr.copy()
                temp[i] = curr[j]
                temp[j] = curr[i]
                new_val = evaluate(temp,size)
                if new_val < val:
                    return new_val, temp
    return val, curr

import csv
fname = "qapdata/bur26a.dat"
# read flash.dat to a list of lists
data = [list(map(int, i.split())) for i in open(fname,'r').readlines()]
for row in data:
    for item in row:
        item = int(item)
size = data[0][0]
flows = data[2:2+size]
distances = data[3+size:len(data)-1]
# print(len(flows),len(distances))
# print(flows)
# print(distances)
current = [i for i in range(size)]
val = evaluate(current, size)
print(val,current)
for i in range(100):
    val, current = local_search_first(val, current, size)
print(val,current)
print(evaluate(current,size))

In [ ]:
import numpy as np

#greedy initialisation taking the pair of highest flows and mathing them to the pair of closest facilities,
# Or if the order is flipped, matching the farthest facilities to the lowest flows

def constructive_init(flow, distance, size):
    assign = np.full(size, -1)
    flownp = np.array(flow)
    distancenp = np.array(distance)
    max_distance_plus1 = distancenp.flatten().max()+1
    for i in range(size):
        distancenp[i][i] = max_distance_plus1
        flownp[i][i] = -1
    for pair in range(size//2): 
        # print(assign)
        # print(flownp)
        # print(distancenp)
        #grab the indices of the max flows and min distances (or vice-versa/dual)
        max_flow_arg_i,max_flow_arg_j = np.unravel_index(flownp.argmax(), flownp.shape)
        min_dist_arg_k,min_dist_arg_p = np.unravel_index(distancenp.argmin(), distancenp.shape)
        # print(max_flow_arg_i,max_flow_arg_j)
        # print(min_dist_arg_k,min_dist_arg_p)
        #set the facility of i to k and of j to p
        assign[max_flow_arg_i] = min_dist_arg_k
        assign[max_flow_arg_j] = min_dist_arg_p
        
        #remove the set facilities and flows by setting max flow to -1 and min dist to max dist+1
        flownp[max_flow_arg_i] = -1
        flownp[max_flow_arg_j] = -1
        flownp[:,max_flow_arg_i] = -1
        flownp[:,max_flow_arg_j] = -1
        distancenp[min_dist_arg_k] = max_distance_plus1
        distancenp[min_dist_arg_p] = max_distance_plus1
        distancenp[:,min_dist_arg_k] = max_distance_plus1
        distancenp[:,min_dist_arg_p] = max_distance_plus1
    #if odd size add final value
    if size%2==1:
        for i in range(size):
            if i not in assign:
                assign[np.where(assign == -1)]=i
                break
    return assign
sol = constructive_init(flows, distances, size)
print(sol)
print(evaluate(sol, size))
#run on "chr12c.dat"
#before using local search 19834 [ 4  6  2  7  3  9  0  5  8 10 11  1]
#after local first search 12816 [ 0  6  2  9  3  7  1  4  8 10 11  5]